# Lesson 7: Advanced Tools, Context & MCP

This interactive notebook teaches advanced tool patterns for production-ready agents:

- **Class-based tools** with shared state
- **ToolContext API** for self-aware tools
- **Invocation state** for request-specific context
- **SlidingWindowConversationManager** for fixed-size history
- **SummarizingConversationManager** for intelligent compression
- **Combined patterns** for production agents

## Learning Objectives

By the end of this lesson, you will be able to:
1. ✅ Create class-based tools that share state through instance attributes
2. ✅ Use ToolContext API to access agent properties and invocation metadata
3. ✅ Manage long conversations with SlidingWindowConversationManager
4. ✅ Compress conversation history intelligently with SummarizingConversationManager
5. ✅ Build production-ready tools combining all these patterns

In [ ]:
# Setup and imports
from datetime import datetime

from lesson_utils import (
    load_environment,
    create_working_model,
    check_api_keys,
    print_troubleshooting
)

from strands import Agent, tool, ToolContext
from strands.agent.conversation_manager import (
    NullConversationManager,
    SlidingWindowConversationManager,
    SummarizingConversationManager
)

# Load environment and check API keys
load_environment()
check_api_keys()

## Part 1: Class-Based Tools (NotebookManager)

Class-based tools allow multiple tools to share state through instance attributes. This is useful when tools need access to common resources like connections, caches, or shared data structures.

**Key concepts:**
- Decorate class methods with `@tool`
- Share state via `self` attributes
- Pass bound methods to the agent

**Reference:** [Class-Based Tools Documentation](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/python-tools/#class-based-tools)

In [ ]:
class NotebookManager:
    """Manage a simple notebook for storing user notes and ideas."""
    
    def __init__(self):
        # Shared state: all tool methods can access these instance attributes
        self.notes = {}  # key: note_id, value: content
        self.next_id = 1
    
    @tool
    def add_note(self, content: str) -> str:
        """Add a new note to the notebook."""
        note_id = self.next_id
        self.notes[note_id] = content
        self.next_id += 1
        return f"Added note #{note_id}: {content[:50]}..."
    
    @tool
    def get_note(self, note_id: int) -> str:
        """Retrieve a specific note by ID."""
        if note_id in self.notes:
            return f"Note #{note_id}: {self.notes[note_id]}"
        return f"Note #{note_id} not found"
    
    @tool
    def list_notes(self) -> str:
        """List all notes in the notebook."""
        if not self.notes:
            return "No notes yet"
        return "\n".join([f"#{id}: {content[:30]}..."
                          for id, content in self.notes.items()])
    
    @tool
    def delete_note(self, note_id: int) -> str:
        """Delete a note from the notebook."""
        if note_id in self.notes:
            del self.notes[note_id]
            return f"Deleted note #{note_id}"
        return f"Note #{note_id} not found"

print("✅ NotebookManager class defined")

In [ ]:
# Test class-based tools
model = create_working_model()
if not model:
    print_troubleshooting()
else:
    # Create an instance of the notebook manager
    notebook = NotebookManager()
    
    # Pass the bound methods as tools to the agent
    agent = Agent(
        model=model,
        tools=[notebook.add_note, notebook.list_notes, notebook.get_note, notebook.delete_note],
        system_prompt="You are a helpful assistant with access to a notebook."
    )
    
    print("📝 Testing notebook with multiple operations...\n")
    
    response1 = agent("Add a note: 'Buy groceries tomorrow'")
    print(f"Response 1: {response1}\n")
    
    response2 = agent("Add another note: 'Call dentist for appointment'")
    print(f"Response 2: {response2}\n")
    
    response3 = agent("What notes do I have?")
    print(f"Response 3: {response3}\n")
    
    print("✅ Class-based tools maintain shared state across invocations!")

## Part 2: ToolContext API - Self-Aware Tools

The ToolContext API allows tools to access information about their execution environment:

- **`tool_context.agent`** - Access agent properties (name, state, messages)
- **`tool_context.tool_use`** - Get current tool invocation metadata (toolUseId, toolName, input)
- **`tool_context.invocation_state`** - Access per-request context data (user_id, api_key, etc.)

**When to use each:**
- **Tool parameters** - Data the LLM should reason about (search queries, file paths, etc.)
- **Invocation state** - Context that affects behavior but shouldn't appear in prompts (user IDs, API keys)
- **Class attributes** - Configuration that doesn't change between requests (connection strings, endpoints)

**Reference:** [ToolContext Documentation](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/python-tools/#toolcontext)

In [ ]:
@tool(context=True)
def log_interaction(message: str, tool_context: ToolContext) -> str:
    """Log an interaction with metadata for audit trail."""
    agent_name = tool_context.agent.name
    tool_id = tool_context.tool_use["toolUseId"]
    timestamp = datetime.now().isoformat()
    
    log_entry = f"[{timestamp}] Agent '{agent_name}' (tool {tool_id}): {message}"
    return f"Logged: {log_entry}"

@tool(context=True)
def send_email(to: str, subject: str, body: str, tool_context: ToolContext) -> str:
    """Send email with user identification for security."""
    user_id = tool_context.invocation_state.get("user_id", "unknown")
    user_email = tool_context.invocation_state.get("user_email", "noreply@example.com")
    
    return f"Email sent from {user_email} (user: {user_id}) to {to}\nSubject: {subject}"

@tool(context=True)
def make_api_call(endpoint: str, tool_context: ToolContext) -> str:
    """Make API call with user-specific authentication."""
    api_key = tool_context.invocation_state.get("api_key")
    user_id = tool_context.invocation_state.get("user_id")
    
    if not api_key:
        return "Error: No API key provided in invocation state"
    
    return f"Called {endpoint} with user {user_id}'s credentials (key: {api_key[:8]}...)"

print("✅ ToolContext tools defined")

In [ ]:
# Test ToolContext API
model = create_working_model()
if not model:
    print_troubleshooting()
else:
    agent = Agent(
        model=model,
        tools=[log_interaction, send_email, make_api_call],
        name="ContextAgent",
        system_prompt="You are a helpful assistant with logging, email, and API capabilities."
    )
    
    print("🔍 Testing self-aware logging tool...\n")
    response1 = agent("Log this message: 'User session started'")
    print(f"Response: {response1}\n")
    
    print("📧 Testing email with user context...\n")
    response2 = agent(
        "Send an email to john@example.com with subject 'Meeting Tomorrow' and body 'See you at 2pm'",
        user_id="user_123",
        user_email="alice@company.com"
    )
    print(f"Response: {response2}\n")
    
    print("🔌 Testing API call with credentials...\n")
    response3 = agent(
        "Make an API call to /users/profile",
        user_id="user_123",
        api_key="sk_live_abc123xyz789"
    )
    print(f"Response: {response3}\n")
    
    print("✅ ToolContext provides access to agent properties, tool metadata, and invocation state!")

## Part 3: SlidingWindowConversationManager

The SlidingWindowConversationManager maintains a fixed number of recent messages, automatically removing older messages when the limit is exceeded.

**Key features:**
- Maintains fixed window size (default: 20 messages)
- Removes oldest messages when window is full
- Cleans up incomplete message sequences
- Can truncate large tool results to save space

**Use cases:**
- Chatbots with cost constraints
- Applications with strict context limits
- Real-time conversational interfaces

**Reference:** [Conversation Management Documentation](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/conversation-management/)

In [ ]:
# Test SlidingWindowConversationManager
model = create_working_model()
if not model:
    print_troubleshooting()
else:
    # Create conversation manager with small window
    conversation_manager = SlidingWindowConversationManager(
        window_size=6,  # Keep only 6 most recent messages
        should_truncate_results=True
    )
    
    notebook = NotebookManager()
    agent = Agent(
        model=model,
        tools=[notebook.add_note, notebook.list_notes],
        conversation_manager=conversation_manager,
        system_prompt="You are a helpful assistant with a notebook."
    )
    
    print(f"📊 Initial message count: {len(agent.messages)}")
    print(f"Window size: 6 messages\n")
    print("💬 Having 8 exchanges (will exceed window size)...\n")
    
    agent("Add note: Meeting at 3pm")
    print(f"   After exchange 1: {len(agent.messages)} messages")
    
    agent("Add note: Buy coffee beans")
    print(f"   After exchange 2: {len(agent.messages)} messages")
    
    agent("Add note: Call Sarah back")
    print(f"   After exchange 3: {len(agent.messages)} messages")
    
    agent("Add note: Review pull request")
    print(f"   After exchange 4: {len(agent.messages)} messages")
    
    agent("Add note: Schedule dentist")
    print(f"   After exchange 5: {len(agent.messages)} messages")
    
    agent("Add note: Pay electricity bill")
    print(f"   After exchange 6: {len(agent.messages)} messages")
    
    agent("Add note: Water plants")
    print(f"   After exchange 7: {len(agent.messages)} messages")
    
    response = agent("What's the most recent note?")
    print(f"   After exchange 8: {len(agent.messages)} messages")
    print(f"\n   Final response: {response}")
    
    print(f"\n✅ Window maintained at ~{len(agent.messages)} messages")
    print("   Older messages were automatically pruned!")

## Part 4: SummarizingConversationManager

The SummarizingConversationManager intelligently compresses conversation history by summarizing older messages instead of simply discarding them.

**Key features:**
- Summarizes older messages to preserve context
- Keeps recent messages intact
- Uses LLM to generate summaries
- Customizable summarization prompts

**Configuration:**
- `summary_ratio` (default: 0.3) - Percentage of messages to summarize
- `preserve_recent_messages` (default: 10) - Number of recent messages to keep
- `summarization_system_prompt` - Custom prompt for summarization
- `summarization_agent` - Custom agent for generating summaries

**Use cases:**
- Long research or debugging sessions
- Extended planning conversations
- When context preservation is critical

**Reference:** [SummarizingConversationManager Documentation](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/conversation-management/#summarizingconversationmanager)

In [ ]:
# Test SummarizingConversationManager
model = create_working_model()
if not model:
    print_troubleshooting()
else:
    # Custom system prompt for technical summarization
    custom_prompt = """Summarize this conversation focusing on:
- Key information the user provided or requested
- Important decisions or preferences expressed
- Action items or tasks discussed
- Technical details like names, numbers, or specifications
Format as concise bullet points in third person."""
    
    conversation_manager = SummarizingConversationManager(
        summary_ratio=0.3,
        preserve_recent_messages=3,
        summarization_system_prompt=custom_prompt
    )
    
    notebook = NotebookManager()
    agent = Agent(
        model=model,
        tools=[notebook.add_note, notebook.list_notes],
        conversation_manager=conversation_manager,
        system_prompt="You are a helpful project planning assistant with a notebook."
    )
    
    print(f"📊 Initial message count: {len(agent.messages)}")
    print("   Summary ratio: 0.3 (30%)")
    print("   Preserve recent: 3 messages\n")
    print("💬 Having extended planning conversation...\n")
    
    agent("I'm planning a new web application project")
    print(f"   After exchange 1: {len(agent.messages)} messages")
    
    agent("Add note: Project name is TaskMaster Pro")
    print(f"   After exchange 2: {len(agent.messages)} messages")
    
    agent("Add note: Target launch date Q2 2025")
    print(f"   After exchange 3: {len(agent.messages)} messages")
    
    agent("Add note: Use React frontend with FastAPI backend")
    print(f"   After exchange 4: {len(agent.messages)} messages")
    
    agent("Add note: PostgreSQL database with Redis cache")
    print(f"   After exchange 5: {len(agent.messages)} messages")
    
    agent("Add note: Deploy on AWS using ECS Fargate")
    print(f"   After exchange 6: {len(agent.messages)} messages")
    
    response = agent("What are the key technical decisions we've made?")
    print(f"   After exchange 7: {len(agent.messages)} messages")
    print(f"\n   Final response: {response}")
    
    print(f"\n✅ Summarization preserves context while managing token usage!")

## Part 5: Combined Pattern - ResearchAssistant

This demonstrates a production-ready pattern combining all concepts:

- **Class-based tools** for shared state (sources, citations)
- **ToolContext** for user tracking and audit logging
- **Invocation state** for user identification
- **Conversation management** for long research sessions

This is a realistic example of how these patterns work together in production agents.

In [ ]:
class ResearchAssistant:
    """Research assistant with source tracking and citation logging."""
    
    def __init__(self):
        self.sources = {}  # Collected research sources
        self.citations = []  # Citation log with timestamps
    
    @tool(context=True)
    def add_source(self, url: str, title: str, summary: str,
                   tool_context: ToolContext) -> str:
        """Add a research source with automatic logging."""
        user_id = tool_context.invocation_state.get("user_id", "anonymous")
        tool_id = tool_context.tool_use["toolUseId"]
        
        self.sources[url] = {
            "title": title,
            "summary": summary,
            "added_by": user_id,
            "tool_id": tool_id,
            "timestamp": datetime.now().isoformat()
        }
        return f"Added source: {title} ({url})"
    
    @tool(context=True)
    def cite_source(self, url: str, tool_context: ToolContext) -> str:
        """Cite a source in your work."""
        if url not in self.sources:
            return f"Source {url} not found. Add it first with add_source."
        
        source = self.sources[url]
        user_id = tool_context.invocation_state.get("user_id", "anonymous")
        
        citation = {
            "url": url,
            "title": source["title"],
            "cited_by": user_id,
            "timestamp": datetime.now().isoformat()
        }
        self.citations.append(citation)
        
        return f"Citation: {source['title']} - {url}"
    
    @tool
    def list_sources(self) -> str:
        """List all research sources."""
        if not self.sources:
            return "No sources added yet"
        
        result = "Research Sources:\n"
        for url, info in self.sources.items():
            result += f"- {info['title']}: {url}\n"
            result += f"  Summary: {info['summary']}\n"
            result += f"  Added by: {info['added_by']}\n"
        return result
    
    @tool
    def get_bibliography(self) -> str:
        """Generate bibliography from citations."""
        if not self.citations:
            return "No citations yet"
        
        result = "Bibliography:\n"
        for i, cite in enumerate(self.citations, 1):
            result += f"{i}. {cite['title']} - {cite['url']}\n"
        return result

print("✅ ResearchAssistant class defined")

In [ ]:
# Test combined pattern
model = create_working_model()
if not model:
    print_troubleshooting()
else:
    research = ResearchAssistant()
    
    agent = Agent(
        model=model,
        tools=[
            research.add_source,
            research.cite_source,
            research.list_sources,
            research.get_bibliography
        ],
        conversation_manager=SlidingWindowConversationManager(window_size=10),
        system_prompt="You are a research assistant helping users collect and cite sources."
    )
    
    print("🔬 Using research assistant with context...\n")
    
    response1 = agent(
        "Add this source: https://arxiv.org/abs/1706.03762 titled 'Attention Is All You Need' "
        "with summary 'Introduces the Transformer architecture for sequence modeling'",
        user_id="researcher_alice"
    )
    print(f"Response 1: {response1}\n")
    
    response2 = agent(
        "Add another source: https://arxiv.org/abs/2005.14165 titled 'GPT-3 Paper' "
        "with summary 'Language models are few-shot learners'",
        user_id="researcher_alice"
    )
    print(f"Response 2: {response2}\n")
    
    response3 = agent(
        "Cite the Attention paper",
        user_id="researcher_alice"
    )
    print(f"Response 3: {response3}\n")
    
    response4 = agent("Generate a bibliography")
    print(f"Response 4: {response4}\n")
    
    print("✅ Combined pattern demonstrates production-ready agent design!")

## ✅ Success Criteria

You've successfully completed Lesson 7 if:

- ☑ **Class-based tools share instance state correctly**
  - NotebookManager maintains notes across multiple tool calls
  - ResearchAssistant maintains sources and citations

- ☑ **ToolContext accesses agent properties**
  - Tools can read `tool_context.agent.name`
  - Tools can access `tool_context.tool_use["toolUseId"]`

- ☑ **Invocation state passes per-request context**
  - Email tool receives user_id and user_email
  - API call tool receives api_key securely
  - Research assistant tracks user_id for each source

- ☑ **SlidingWindowConversationManager maintains window size**
  - Message count stays around configured window_size
  - Older messages automatically removed

- ☑ **SummarizingConversationManager preserves context**
  - Long conversations compressed with summaries
  - Key information preserved despite compression

- ☑ **Combined pattern demonstrates production design**
  - All concepts working together seamlessly
  - Realistic agent use case implemented

## 🧪 Experiments to Try

### Beginner
1. **ContactManager** - Build a class with add/search/update/delete contacts
2. **Usage Tracker** - Create self-aware tool that tracks its own invocation count
3. **Custom Summarization** - Add custom prompt optimizing for code discussions

### Intermediate
4. **BookmarkManager** - Build class with tags, categories, and search capabilities
5. **Multi-Tenant Tool** - Use invocation_state for multi-user/multi-tenant applications
6. **Window Comparison** - Test SlidingWindow vs Summarizing with 50+ turn conversation

### Advanced
7. **FileOrganizer** - Class with categorize/move/search/tag methods
8. **Hybrid Manager** - Custom conversation manager combining window + summarization
9. **Production Agent** - Build complete agent with auth, logging, and conversation management

### Tips
- Copy cells and modify them for experiments
- Add markdown cells to document your findings
- Compare different window sizes and summary ratios
- Try different summarization prompts for various domains